In [ ]:
# 学習済みのベクトルデータの準備
import os
import gdown
if not os.path.exists('/content/data'):
    os.mkdir('/content/data')

url = 'https://drive.google.com/uc?id=1BwRe4FnPi26qzpUZ1r98x9PVkWLaA1d6'
gdown.download(url, './data/stem_vec.tar.gz', quiet=False)
!tar zxvf /content/data/stem_vec.tar.gz


In [ ]:
# 類義語を検索してみる
import gensim
import warnings
from gensim.models import Word2Vec

if not os.path.exists('/content/stem_vec/word2vec.gensim.egotw.model'):
    raise ValueError("ERROR: 学習済みword2vecモデルが存在しません。")

# 学習済みWord2vecを読込
wv = Word2Vec.load("/content/stem_vec/word2vec.gensim.egotw.model")

# 検索
input_word = '学生'
if not input_word in wv:
    print("Input word [%s] is not included in the model" % input_word)
else:
    results = wv.most_similar(positive=[input_word])
    for result in results:
        print(result)

In [ ]:
# word2vecの学習を追加で行う

# 形態素解析器MeCabのラッパーライブラリfugashiをインストール
!pip install fugashi[unidic-lite]
from fugashi import Tagger

txt = '私は徳島大学の学生です。'

tagger = Tagger('-Owakati')

# 形態素解析する関数
def wakatigaki(txt):
    words = []
    wakati = tagger.parse(txt)
    for w in wakati.split(' '):
        words.append(w)
    return words

for w in wakatigaki(txt):
    print(w)

In [ ]:
# コーパスの準備
# テキストデータ(livedoor news corpus)をMeCab(fugashi)で分かち書き処理
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar zxvf ldcc-20140209.tar.gz
# テキストを1つのファイルにまとめる(すべてのファイルで3行目以降が本文)
import glob
wf = open('./all.txt', 'w')
for dir in glob.glob('/content/text/*'):
    for path in glob.glob(f'{dir}/*.txt'):
        l = 0
        for line in open(path):
            line = line.rstrip()
            if line == '':
                continue

            l += 1
            if l >= 3:
                wf.write(' '.join(wakatigaki(line)) + '\n')
wf.close


In [6]:
# livedoor news corpusの分かち書きテキストを
# 追加で学習させる
from gensim.models import word2vec
corpus = '/content/all.txt'
newmodel = '/content/stem_vec/w2v.new.livedoor.model'
sentences = word2vec.Text8Corpus(corpus)

model = word2vec.Word2Vec.load("/content/stem_vec/word2vec.gensim.egotw.model")
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=model.corpus_count, epochs=10)
model.save(newmodel)

In [ ]:
# wrimeコーパスをダウンロード
!git clone https://github.com/ids-cv/wrime

In [94]:
# 文書ベクトルを作成する
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd


# 単語分散表現を格納したリストを作成する
def make_vec(fn, model):
    vlist = []
    for word in fn:
        if not word in model:
            vlist.append(np.array([0.0]*50))
        else:
            vlist.append(model[word])
    vlist = np.asarray(vlist)
    return vlist

# コーパス中の各文の感情ラベルを取得する
def getEmotionLabels( df ):
    cols = ['Writer_Joy', 'Writer_Sadness',
            'Writer_Anticipation','Writer_Surprise',
            'Writer_Anger', 'Writer_Fear', 'Writer_Disgust', 'Writer_Trust']
    emotions = ['喜び', '悲しみ', '期待', '驚き', '怒り', '怖れ', '嫌悪', '信頼']
    emos = []
    for ev in df[cols].values:
        emos.append( emotions[np.argmax(ev)] )
    return emos

# 類似度の閾値（threshold）を設定して，
# 閾値以上のものだけを辞書に格納する
def filtering_result(sims, threshold):
    sh = {}
    for i, s in enumerate(sims):
        if s >= threshold:
            sh[i] = s
    return sh


# メイン関数
def main():

    # 作成済みのlivedoorの分散表現のモデルを読み込んでおく
    livedoor_model_path = '/content/stem_vec/w2v.new.livedoor.model'
    livedoor_model = word2vec.Word2Vec.load(livedoor_model_path)

    # 検索対象となるテキストコーパスを読み込む
    df = pd.read_csv('./wrime/wrime-ver1.tsv', sep='\t')
    df = df[df['Train/Dev/Test'] == 'test']
    sents = df['Sentence'].values
    emos = getEmotionLabels(df)

    print(len(sents))

    cps = []
    with open('./wrime.txt', 'w') as wf:
        for s in sents:
            wl = wakatigaki(s)
            sen = ' '.join(wl)
            wf.write(f'{sen}\n')
            cps.append(sen)

    # コーパスからTF-IDFを計算
    print(len(cps))
    vect = TfidfVectorizer(analyzer=lambda x: x.split(" "), dtype=np.float32, token_pattern="(?u)\\b\\w+\\b")
    wv = vect.fit_transform(cps)
    fn = vect.get_feature_names()

    vlist = make_vec(fn, livedoor_model)

    vsm = wv.sum(axis=1)
    docvec = (wv @ vlist) / vsm

    waka = wakatigaki('今日は全身がだるくて，何もやる気が起きません！')
    avgv = np.array([0.0]*50)
    count = 0.0
    for w in waka:
        if w in model:
            if np.sum(avgv) == 0:
                avgv = livedoor_model[w].copy()
            else:
                avgv += livedoor_model[w].copy()
            count += 1.0

    # 入力されたクエリと検索対象文書集合間の類似度を計算
    query = np.array([avgv])
    query = np.nanmean(query, axis=0).reshape(1, -1)
    sims = cosine_sim(docvec, query)
    # 類似度が0.5以上の結果だけを取得する
    sh = filtering_result(sims, 0.5)

    sh = {}
    for i, s in enumerate(sims):
        if s >= 0.5:
            sh[i] = s

    topk = 30 # 上位30件のみ表示
    emok = {'喜び':0, '悲しみ':0, '期待':0, '驚き':0, '怒り':0, '怖れ':0, '嫌悪':0, '信頼':0}
    for k,v in sorted(sh.items(), key=lambda x:x[1], reverse=True)[:topk]:
        print("SID:[%d], Sim:[%.3lf] <%s> ==> [%s]" % (k,v, cps[k], emos[k]) )
        emok[emos[k]] += 1

    print(emok)


if __name__ == '__main__':
    main()
    

2000
2000


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: DeprecationWarning: Cal

SID:[1048], Sim:[0.890] <今日 は 子ども ら が 早く 寝 た の で 勉強 しよ と 思っ た が 最近 の 無理 の 反動 で 全く 何 も やる気 が 起き ない> ==> [怖れ]
SID:[1169], Sim:[0.838] <久し ぶり に いい 感じ の 眠気 が 。 今日 は 寝よう か な 。 嫌 な こと は 忘れる 。 嫌 な こと 特に ない けど 。> ==> [喜び]
SID:[1976], Sim:[0.836] <いつ も 通り に 筋トレ は し てる けど 、 九州 大雨 が 気 に なっ て 仕方 が ない 。 早く 雨 が やん で くれる 事 を 祈る しか ない 。 無力 さ を 痛感 し ます 。 皆 さん どう か ご 無事 で い て ください !> ==> [悲しみ]
SID:[346], Sim:[0.831] <もう 秋 だ って の に 、 暑く て 汗 かい てる の 私 だけ ？> ==> [悲しみ]
SID:[510], Sim:[0.830] <いっつ も こう やっ て 何 かしら ぬけ落ち て て 悲しく なる 。 (°_°)> ==> [悲しみ]
SID:[1602], Sim:[0.826] <一 度 子供 達 と 寝 落ち し て しまう と 、 その 後 起き て も ずっと 頭痛 が ・ ・ ・ 。 何 で だろう ？ ？ ？ 謎 。> ==> [怖れ]
SID:[408], Sim:[0.820] <と いう か 、 今日 めっちゃ 暑く ない ？> ==> [驚き]
SID:[699], Sim:[0.818] <で も ぜーんぶ 終わっ た 一 週間 と ちょっと 後 は 東京 ！ その 時 に は ストレス フリー に なっ てる はず ！ かぜ なおす ！> ==> [喜び]
SID:[1916], Sim:[0.818] <トレーニング を 始め て から 身体 が 変わる 。 最近 職場 で 「 どう やっ たら 痩せれ た ん ？ 」 と 聞か れる こと が 増え た 。 いや 、 聞か なく て も ホント は みんな わかっ てる 。 やる しか ない 事 は すでに 頭 で わかっ て いる 。 取り組み 始める か どう か 、 それ